In [2]:
import numpy as np
from jax import random
import jax.numpy as jnp
from pathlib import Path
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive
from numpyro.ops.indexing import Vindex
from tqdm import tqdm
from numpyro.handlers import mask

import json

In [3]:
def get_class_probabilities(logits_path, tokens_json_path, label_tokens=["0", "1"]):
    """
    Computes softmax probabilities for class tokens from LLM logits.

    Args:
        logits_path (str): Path to .npy file containing logits (N x top_k).
        tokens_json_path (str): Path to JSON file mapping logits columns to tokens.
        label_tokens (list): List of class token strings, e.g. ["0", "1"].

    Returns:
        np.ndarray: Array of shape (N, 2) with probabilities for class "0" and "1".
    """
    # Load data
    logits = np.load(logits_path)
    with open(tokens_json_path, "r") as f:
        token_map = json.load(f)

    # Find which columns in logits correspond to the class tokens
    label_columns = []
    for pos_str, (token, _) in token_map.items():
        if token.strip() in label_tokens:
            label_columns.append(int(pos_str))

    if len(label_columns) != 2:
        raise ValueError(f"Expected 2 class tokens, found {len(label_columns)}: {label_columns}")

    # Extract relevant logits
    selected_logits = logits[:, label_columns]

    # Softmax function
    def softmax(x):
        e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return e_x / e_x.sum(axis=1, keepdims=True)

    # Calculate probabilities
    probs = softmax(selected_logits)

    return selected_logits, probs

logits_train, probs_train = get_class_probabilities("outputs/train/logits.npy", "outputs/train/top_tokens.json")
logits_val, probs_val = get_class_probabilities("outputs/val/logits.npy", "outputs/val/top_tokens.json")

# Show first 5 examples
for i in range(5):
    print(f"Example {i+1}: P(0) = {probs_train[i, 0]:.3f}, P(1) = {probs_train[i, 1]:.3f}")

print(probs_train[:5])
print(logits_train[:5])

Example 1: P(0) = 0.998, P(1) = 0.002
Example 2: P(0) = 0.998, P(1) = 0.002
Example 3: P(0) = 0.980, P(1) = 0.020
Example 4: P(0) = 0.005, P(1) = 0.995
Example 5: P(0) = 0.626, P(1) = 0.374
[[0.99847525 0.00152479]
 [0.9979493  0.00205074]
 [0.97997653 0.02002344]
 [0.00530189 0.99469805]
 [0.6261242  0.37387583]]
[[27.625    21.140625]
 [26.46875  20.28125 ]
 [26.703125 22.8125  ]
 [22.90625  28.140625]
 [24.015625 23.5     ]]


In [4]:
# --- DATA LOADING HELPERS --- #

def load_jsonl(file_path, max_items=None):
    data = []
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if max_items is not None and i >= max_items:
                break
            data.append(eval(line))
    return data

def create_annotator_mapping(data):
    from collections import defaultdict
    annotator_positions = defaultdict(set)
    for item in data:
        for pos, ann in enumerate(item['annotators']):
            annotator_positions[ann].add(pos)
    annotator_to_positions = {}
    current_position = 0
    for annotator in sorted(annotator_positions.keys()):
        positions = sorted(annotator_positions[annotator])
        for pos in positions:
            annotator_to_positions[(annotator, pos)] = current_position
            current_position += 1
    return annotator_to_positions

def process_annotations(data, annotator_mapping=None):
    if annotator_mapping is None:
        annotator_mapping = create_annotator_mapping(data)

    total_positions = max(annotator_mapping.values()) + 1
    positions = np.zeros(total_positions, dtype=int)
    annotations = np.zeros((len(data), total_positions), dtype=int)
    masks = np.zeros((len(data), total_positions), dtype=bool)

    for item_idx, item in enumerate(data):
        for pos, (annotator, label) in enumerate(zip(item['annotators'], item['labels'])):
            if (annotator, pos) in annotator_mapping:
                matrix_pos = annotator_mapping[(annotator, pos)]
                annotations[item_idx, matrix_pos] = label
                masks[item_idx, matrix_pos] = True
                positions[matrix_pos] = annotator
    return positions, annotations, masks

# as result, matrix annotations with columns meaning (annotator, position) and columns meaning labels for each item on (annotator, position)
# positions[i] - who annotated the column i in annotations

In [6]:
# --- DAWID-SKENE MODEL --- #

def dawid_skene(positions, annotations, masks, use_llm_prior=False, llm_probs=None):
    num_annotators = int(np.max(positions)) + 1
    num_classes = int(np.max(annotations)) + 1
    num_items, num_positions = annotations.shape

    with numpyro.plate("annotator", num_annotators, dim=-2):
        with numpyro.plate("class", num_classes):
            beta = numpyro.sample("beta", dist.Dirichlet(jnp.ones(num_classes)))
    
    if use_llm_prior:
        assert llm_probs is not None, "LLM probabilities must be provided if use_llm_prior is True"
        # pi = numpyro.sample("pi", dist.Dirichlet(llm_probs))
        #pi = jnp.asarray(llm_probs)
        pi = jnp.array(llm_probs[:,np.newaxis,:])  # shape: (num_items, num_classes)

    else:
        pi = numpyro.sample("pi", dist.Dirichlet(jnp.ones(num_classes)))

    with numpyro.plate("item", num_items, dim=-2):
        c = numpyro.sample("c", dist.Categorical(probs=pi), infer={"enumerate": "parallel"})

        with numpyro.plate("position", num_positions):
            with mask(mask=masks):
                numpyro.sample(
                    "y",
                    dist.Categorical(Vindex(beta)[positions, c, :]),
                    obs=annotations,
                )


# --- MAIN EXECUTION --- #

def run_ds_on_subset(json_path, use_llm_prior=False, llm_probs=None, max_items=None):
    data = load_jsonl(json_path, max_items=max_items)
    positions, annotations, masks = process_annotations(data)

    if use_llm_prior:
        if llm_probs is None:
            raise ValueError("llm_probs must be provided when use_llm_prior=True")
        if len(llm_probs.shape) != 2:
            raise ValueError(f"llm_probs should be 2D array, got shape {llm_probs.shape}")
        if llm_probs.shape[0] < annotations.shape[0]:
            raise ValueError(f"Not enough LLM probabilities for all items: {llm_probs.shape[0]} < {annotations.shape[0]}")
        llm_probs = llm_probs[:annotations.shape[0]]
    
    kernel = NUTS(dawid_skene)
    mcmc = MCMC(kernel, num_warmup=500, num_samples=1000)
    mcmc.run(
        random.PRNGKey(0),
        positions,
        annotations,
        masks,
        use_llm_prior=use_llm_prior,
        llm_probs=llm_probs,
    )
    mcmc.print_summary()

    samples = mcmc.get_samples()
    beta_mean = jnp.mean(samples['beta'], axis=0)

    print("\nInferred confusion matrices (beta) for annotators:")
    for i, matrix in enumerate(beta_mean):
        print(f"Annotator {i}:\n{np.round(matrix, 2)}\n")

    predictive = Predictive(dawid_skene, samples, infer_discrete=True)
    discrete_samples = predictive(
        random.PRNGKey(1),
        positions,
        annotations,
        masks,
        use_llm_prior=use_llm_prior,
        llm_probs=llm_probs,
    )
    predicted_labels = discrete_samples["c"]
    return samples, predicted_labels, beta_mean

# if __name__ == "__main__":
#    mcmc_samples, predicted_labels, beta_mean = run_ds_on_subset("data/ghc_train.jsonl", use_llm_prior=True, llm_probs=probs_train)

In [8]:
from numpyro.infer import Predictive

# train_data = load_jsonl("data/ghc_train.jsonl", max_items=100)
# train_positions, train_annotations, train_masks = process_annotations(train_data)

val_data = load_jsonl("data/ghc_val.jsonl", max_items=100)
val_positions, val_annotations, val_masks = process_annotations(val_data)

train_mcmc_samples, predicted_labels_train, beta_mean_train = run_ds_on_subset(
    json_path="data/ghc_train.jsonl",
    use_llm_prior=True,
    llm_probs=probs_train,
    max_items=100
)

predictive_val = Predictive(
    dawid_skene,
    posterior_samples={"beta": beta_mean_train[None, ...]},  # making shape (1, annotators, C, C)
    infer_discrete=True
)

probs_val = probs_val[:val_annotations.shape[0]]

val_pred = predictive_val(
    random.PRNGKey(1),
    val_positions,
    val_annotations,
    val_masks,
    use_llm_prior=True,
    llm_probs=probs_val[:val_annotations.shape[0]]
)

val_predicted_labels = np.array(val_pred["c"]).squeeze()  # shape (num_val_items,)

sample: 100%|██████████| 1500/1500 [00:03<00:00, 469.08it/s, 7 steps of size 3.39e-01. acc. prob=0.92] 



                  mean       std    median      5.0%     95.0%     n_eff     r_hat
 beta[0,0,0]      0.83      0.12      0.85      0.67      1.00   1557.15      1.00
 beta[0,0,1]      0.17      0.12      0.15      0.00      0.33   1557.15      1.00
 beta[0,1,0]      0.30      0.16      0.28      0.02      0.54   1674.09      1.00
 beta[0,1,1]      0.70      0.16      0.72      0.46      0.98   1674.09      1.00
 beta[1,0,0]      0.84      0.14      0.89      0.64      1.00   1680.09      1.00
 beta[1,0,1]      0.16      0.14      0.11      0.00      0.36   1680.09      1.00
 beta[1,1,0]      0.78      0.18      0.83      0.51      1.00   1747.06      1.00
 beta[1,1,1]      0.22      0.18      0.17      0.00      0.49   1747.06      1.00
 beta[2,0,0]      0.83      0.14      0.87      0.64      1.00   1695.99      1.00
 beta[2,0,1]      0.17      0.14      0.13      0.00      0.36   1695.99      1.00
 beta[2,1,0]      0.51      0.23      0.52      0.15      0.89   1571.79      1.00
 be

In [9]:
from numpyro.infer import Predictive
import numpy as np # Ensure numpy is imported if used for post-processing c_val_samples
from jax import random # For random.PRNGKey

# val_data, val_positions, val_annotations, val_masks are loaded and processed correctly
val_data = load_jsonl("data/ghc_val.jsonl", max_items=100)
val_positions, val_annotations, val_masks = process_annotations(val_data)

# Assume probs_train and probs_val are loaded and processed from previous cells
# Ensure probs_train is correctly sliced if its length doesn't match max_items for run_ds_on_subset

# Call run_ds_on_subset for training
# The 'max_items=100' argument to run_ds_on_subset implies that
# your run_ds_on_subset function should be able to pass this to its internal load_jsonl call.
# If not, adjust run_ds_on_subset or ensure probs_train aligns with its default loading behavior.
train_mcmc_samples, predicted_labels_train_point_estimate, beta_mean_train = run_ds_on_subset(
    json_path="data/ghc_train.jsonl",
    use_llm_prior=True,
    llm_probs=probs_train, # Make sure probs_train corresponds to the items loaded by json_path (e.g., first 100)
    max_items=100 # As in your selection, ensure run_ds_on_subset handles this
)
# predicted_labels_train_point_estimate is based on all samples from this MCMC run itself.

# --- Start of changes for validation prediction ---

# Create Predictive to get samples of 'c' on the validation set,
# using ALL beta samples obtained during training.
predictive_for_val_c_samples = Predictive(
    dawid_skene,
    posterior_samples={'beta': train_mcmc_samples['beta']}, # Use all MCMC beta samples
    infer_discrete=True,
    return_sites=['c']  # Specify that we want samples for site 'c'
)

# Prepare probs_val for validation:
# Ensure the correct number of items and 2D shape.
num_val_items_loaded = val_annotations.shape[0]
probs_val_for_pred = probs_val[:num_val_items_loaded]

# If probs_val_for_pred happens to be 3D (N, 1, C), squeeze to 2D (N, C)
if probs_val_for_pred.ndim == 3 and probs_val_for_pred.shape[1] == 1:
    probs_val_for_pred = probs_val_for_pred.squeeze(axis=1)
elif probs_val_for_pred.ndim != 2:
    # If shape is still not 2D, there's an issue with probs_val preparation
    raise ValueError(f"probs_val_for_pred has an unexpected shape: {probs_val_for_pred.shape}, expected 2D.")

# Get predictive samples for 'c' for the validation set
val_c_dist_output = predictive_for_val_c_samples(
    random.PRNGKey(1),  # Can use a different key if needed for reproducibility
    val_positions,
    val_annotations,
    val_masks,
    use_llm_prior=True,
    llm_probs=probs_val_for_pred # Pass the prepared probs_val
)

# c_val_samples will have shape (num_mcmc_beta_samples, num_validation_items)
c_val_samples = val_c_dist_output["c"]

# --- End of changes ---

# The old code for val_predicted_labels based on beta_mean_train is now replaced by c_val_samples.
# If a single array of predicted labels is still needed, it can be derived from c_val_samples, e.g.:
# val_predicted_labels_mode = np.array([np.argmax(np.bincount(c_val_samples[:, i].astype(np.int32))) for i in range(c_val_samples.shape[1])])
# print(f"Example val_predicted_labels (mode from c_val_samples): {val_predicted_labels_mode[:10]}")

print(f"Shape of beta samples from training (train_mcmc_samples['beta']): {train_mcmc_samples['beta'].shape}")
print(f"Shape of c_val_samples (predicted 'c' for validation): {c_val_samples.shape}")

sample: 100%|██████████| 1500/1500 [00:03<00:00, 472.94it/s, 7 steps of size 3.39e-01. acc. prob=0.92] 



                  mean       std    median      5.0%     95.0%     n_eff     r_hat
 beta[0,0,0]      0.83      0.12      0.85      0.67      1.00   1557.15      1.00
 beta[0,0,1]      0.17      0.12      0.15      0.00      0.33   1557.15      1.00
 beta[0,1,0]      0.30      0.16      0.28      0.02      0.54   1674.09      1.00
 beta[0,1,1]      0.70      0.16      0.72      0.46      0.98   1674.09      1.00
 beta[1,0,0]      0.84      0.14      0.89      0.64      1.00   1680.09      1.00
 beta[1,0,1]      0.16      0.14      0.11      0.00      0.36   1680.09      1.00
 beta[1,1,0]      0.78      0.18      0.83      0.51      1.00   1747.06      1.00
 beta[1,1,1]      0.22      0.18      0.17      0.00      0.49   1747.06      1.00
 beta[2,0,0]      0.83      0.14      0.87      0.64      1.00   1695.99      1.00
 beta[2,0,1]      0.17      0.14      0.13      0.00      0.36   1695.99      1.00
 beta[2,1,0]      0.51      0.23      0.52      0.15      0.89   1571.79      1.00
 be

In [11]:
# New cell for evaluating annotator response prediction accuracy
import jax.numpy as jnp # Ensure jax.numpy is imported

# Check if necessary variables are defined (assuming they are from previous cells)
if 'c_val_samples' not in globals() or 'beta_mean_train' not in globals():
    print("Error: 'c_val_samples' or 'beta_mean_train' not found.")
    print("Please ensure you have run the predictions for 'c' on the validation set ")
    print("and have the 'beta_mean_train' from the training phase.")
else:
    num_mcmc_draws, num_val_items = c_val_samples.shape
    # num_annotators, num_classes, _ = beta_mean_train.shape # Should be defined from beta_mean_train

    # Ensure val_annotations, val_positions, val_masks are also available from previous cells
    if 'val_annotations' not in globals() or 'val_positions' not in globals() or 'val_masks' not in globals():
        print("Error: Validation annotation data (val_annotations, val_positions, val_masks) not found.")
    elif val_annotations.shape[0] != num_val_items:
         raise ValueError(f"Number of items in c_val_samples ({num_val_items}) "
                          f"does not match val_annotations ({val_annotations.shape[0]})")
    else:
        num_val_items_from_annot, num_positions_in_matrix = val_annotations.shape

        correct_predictions_count = 0
        total_predictions_made = 0

        print(f"Calculating accuracy of predicting annotator responses...")
        print(f"Using {num_mcmc_draws} MCMC samples for c_val.")

        # Convert to NumPy for the outer loop if jax.jit is not used for the whole block,
        # or keep as JAX arrays if the whole computation can be JITted.
        # For simplicity here, direct iteration is shown.
        
        for mcmc_idx in range(num_mcmc_draws):
            if (mcmc_idx + 1) % 100 == 0: # Print progress every 100 samples
                 print(f"  Processed MCMC samples: {mcmc_idx + 1}/{num_mcmc_draws}")

            for item_idx in range(num_val_items):
                # Get the 'true' label for this item from the current MCMC sample of c
                c_true_sample_for_item = c_val_samples[mcmc_idx, item_idx]

                for pos_idx in range(num_positions_in_matrix):
                    if val_masks[item_idx, pos_idx]: # If there's an actual annotation
                        annotator_id = val_positions[pos_idx]
                        actual_annotator_label = val_annotations[item_idx, pos_idx]

                        # Probability distribution of this annotator's response P(y_ann | c_true, beta_ann)
                        # This is beta_mean_train[annotator_id, c_true_sample_for_item, :]
                        prob_dist_annotator_response = beta_mean_train[annotator_id, c_true_sample_for_item, :]

                        # Make a hard prediction for this annotator's response
                        predicted_annotator_label = jnp.argmax(prob_dist_annotator_response)

                        if predicted_annotator_label == actual_annotator_label:
                            correct_predictions_count += 1
                        total_predictions_made += 1
        
        print("Calculation finished.")

        if total_predictions_made > 0:
            accuracy_of_predicting_annotator_responses = correct_predictions_count / total_predictions_made
            print(f"\nAccuracy of predicting annotator responses on validation: {accuracy_of_predicting_annotator_responses:.4f}")
            print(f"(Based on {total_predictions_made} individual annotator response predictions)")
        else:
            print("\nNo predictions were made for annotator responses.")
            print("Check val_masks (perhaps all False) or the number of MCMC samples.")


ValueError: too many values to unpack (expected 2)